# 52-异步任务队列

## 📚 用途说明

**学习目标**：
- 掌握异步任务队列的核心概念和实现
- 学会任务优先级管理和调度策略
- 理解任务重试机制和错误处理
- 能够构建分布式任务队列系统

**前置要求**：
- 已完成并发编程模式学习
- 熟练掌握生产者消费者模式
- 了解基本的队列数据结构
- 理解异步编程和事件循环机制

**与LangChain关联**：
- 任务队列优化LangChain的批量处理能力
- 优先级管理提升LangChain的任务调度效率
- 重试机制确保LangChain的可靠性
- 分布式队列支持LangChain的水平扩展

---

## 🔢 知识点覆盖

### 6.7 异步任务队列 [⭐⭐进阶]
**知识点说明**：异步任务队列是构建高性能、可靠异步系统的核心技术，对于LangChain的大规模任务处理和分布式部署具有重要意义。

**学习要求**：
- 掌握异步任务队列的核心概念和实现
- 学会任务优先级管理和调度策略
- 理解任务重试机制和错误处理
- 能够构建分布式任务队列系统

**案例要求**：
- 实现完整的异步任务队列系统
- 进行任务队列性能测试和优化
- 应用任务队列解决实际问题
- 验证点：能独立设计和实现任务队列解决方案

In [ ]:
print("📋 异步任务队列:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import uuid
import pickle
import hashlib
from typing import List, Dict, Any, Optional, Callable, Union
from dataclasses import dataclass, asdict, field
from enum import Enum
from collections import deque
import weakref

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 异步任务队列基础
print(f"📝 1. 异步任务队列基础:")

# 1.1 任务状态和类型定义
print(f"\n   🔍 1.1 任务状态和类型定义:")

class TaskStatus(Enum):
    """任务状态枚举"""
    PENDING = "pending"      # 等待执行
    RUNNING = "running"      # 正在执行
    COMPLETED = "completed"  # 执行完成
    FAILED = "failed"        # 执行失败
    RETRYING = "retrying"    # 重试中
    CANCELLED = "cancelled"  # 已取消

class TaskPriority(Enum):
    """任务优先级枚举"""
    LOW = 3      # 低优先级
    NORMAL = 2   # 普通优先级
    HIGH = 1     # 高优先级
    URGENT = 0   # 紧急优先级

@dataclass
class AsyncTask:
    """异步任务"""
    id: str
    func: Callable
    args: tuple = field(default_factory=tuple)
    kwargs: dict = field(default_factory=dict)
    priority: TaskPriority = TaskPriority.NORMAL
    max_retries: int = 3
    timeout: float = 30.0
    created_at: float = field(default_factory=time.time)
    scheduled_at: float = field(default_factory=time.time)
    
    # 运行时状态
    status: TaskStatus = TaskStatus.PENDING
    retry_count: int = 0
    started_at: Optional[float] = None
    completed_at: Optional[float] = None
    result: Any = None
    error: Optional[str] = None
    worker_id: Optional[str] = None
    
    def __post_init__(self):
        if not self.id:
            self.id = str(uuid.uuid4())
    
    def to_dict(self) -> Dict[str, Any]:
        """转换为字典（用于序列化）"""
        data = asdict(self)
        # 转换枚举为字符串
        data['priority'] = self.priority.value
        data['status'] = self.status.value
        # 函数对象无法序列化，存储函数名
        data['func_name'] = self.func.__name__ if hasattr(self.func, '__name__') else str(self.func)
        data.pop('func', None)
        return data
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any], func: Callable) -> 'AsyncTask':
        """从字典创建任务"""
        # 转换字符串为枚举
        data['priority'] = TaskPriority(data['priority'])
        data['status'] = TaskStatus(data['status'])
        data['func'] = func
        data.pop('func_name', None)
        return cls(**data)

@dataclass
class TaskResult:
    """任务结果"""
    task_id: str
    status: TaskStatus
    result: Any = None
    error: Optional[str] = None
    execution_time: float = 0.0
    worker_id: Optional[str] = None
    completed_at: float = field(default_factory=time.time)

print(f"   ✅ 任务状态和类型定义完成")
print(f"      - TaskStatus: {', '.join([s.value for s in TaskStatus])}")
print(f"      - TaskPriority: {', '.join([p.name for p in TaskPriority])}")
print(f"      - AsyncTask: 异步任务数据结构")
print(f"      - TaskResult: 任务结果数据结构")

# 1.2 基础异步任务队列
print(f"\n   🏗️ 1.2 基础异步任务队列:")

class AsyncTaskQueue:
    """基础异步任务队列"""
    
    def __init__(self, max_queue_size: int = 1000, max_workers: int = 5):
        self.max_queue_size = max_queue_size
        self.max_workers = max_workers
        
        # 优先级队列
        self.task_queue = asyncio.PriorityQueue(maxsize=max_queue_size)
        self.result_queue = asyncio.Queue(maxsize=max_queue_size)
        
        # 任务存储
        self.tasks: Dict[str, AsyncTask] = {}
        self.task_results: Dict[str, TaskResult] = {}
        
        # 工作进程
        self.workers: List[asyncio.Task] = []
        self.running = False
        
        # 统计信息
        self.stats = {
            "total_tasks": 0,
            "completed_tasks": 0,
            "failed_tasks": 0,
            "retry_count": 0,
            "total_execution_time": 0.0
        }
    
    async def start(self):
        """启动任务队列"""
        if self.running:
            return
        
        self.running = True
        
        # 启动工作进程
        self.workers = [
            asyncio.create_task(self._worker(f"worker-{i}"))
            for i in range(self.max_workers)
        ]
        
        print(f"      🚀 异步任务队列启动 ({self.max_workers} 个工作进程)")
    
    async def stop(self):
        """停止任务队列"""
        if not self.running:
            return
        
        self.running = False
        
        # 取消所有工作进程
        for worker in self.workers:
            worker.cancel()
        
        # 等待工作进程完成
        await asyncio.gather(*self.workers, return_exceptions=True)
        
        print(f"      🛑 异步任务队列停止")
    
    async def submit(self, func: Callable, *args, 
                     priority: TaskPriority = TaskPriority.NORMAL,
                     max_retries: int = 3, 
                     timeout: float = 30.0,
                     **kwargs) -> str:
        """提交任务到队列"""
        task = AsyncTask(
            id=str(uuid.uuid4()),
            func=func,
            args=args,
            kwargs=kwargs,
            priority=priority,
            max_retries=max_retries,
            timeout=timeout
        )
        
        try:
            await self.task_queue.put((task.priority.value, task.id))
            self.tasks[task.id] = task
            self.stats["total_tasks"] += 1
            
            print(f"      📤 任务提交: {task.id} (优先级: {task.priority.name})")
            return task.id
            
        except asyncio.QueueFull:
            raise Exception("任务队列已满")
    
    async def get_result(self, task_id: str, timeout: float = None) -> Optional[TaskResult]:
        """获取任务结果"""
        # 首先检查是否已有结果
        if task_id in self.task_results:
            return self.task_results[task_id]
        
        # 等待结果
        try:
            if timeout:
                result = await asyncio.wait_for(self.result_queue.get(), timeout=timeout)
            else:
                result = await self.result_queue.get()
            
            self.task_results[result.task_id] = result
            
            if result.task_id == task_id:
                return result
            else:
                # 不是我们要的结果，放回队列
                await self.result_queue.put(result)
                return None
                
        except asyncio.TimeoutError:
            return None
    
    async def cancel_task(self, task_id: str) -> bool:
        """取消任务"""
        if task_id in self.tasks:
            task = self.tasks[task_id]
            if task.status in [TaskStatus.PENDING, TaskStatus.RETRYING]:
                task.status = TaskStatus.CANCELLED
                print(f"      ❌ 任务取消: {task_id}")
                return True
        return False
    
    async def _worker(self, worker_id: str):
        """工作进程"""
        print(f"      👷 {worker_id} 开始工作")
        
        while self.running:
            try:
                # 获取任务
                priority, task_id = await asyncio.wait_for(
                    self.task_queue.get(), 
                    timeout=1.0
                )
                
                if task_id not in self.tasks:
                    continue
                
                task = self.tasks[task_id]
                
                # 检查任务状态
                if task.status == TaskStatus.CANCELLED:
                    continue
                
                # 检查调度时间
                if task.scheduled_at > time.time():
                    # 重新放回队列
                    await self.task_queue.put((task.priority.value, task.id))
                    await asyncio.sleep(0.1)
                    continue
                
                # 执行任务
                await self._execute_task(task, worker_id)
                
            except asyncio.TimeoutError:
                continue
            except Exception as e:
                print(f"      ❌ {worker_id} 工作进程错误: {e}")
        
        print(f"      👷 {worker_id} 停止工作")
    
    async def _execute_task(self, task: AsyncTask, worker_id: str):
        """执行任务"""
        task.status = TaskStatus.RUNNING
        task.started_at = time.time()
        task.worker_id = worker_id
        
        try:
            # 执行任务
            if asyncio.iscoroutinefunction(task.func):
                result = await asyncio.wait_for(
                    task.func(*task.args, **task.kwargs),
                    timeout=task.timeout
                )
            else:
                # 同步函数在线程池中执行
                loop = asyncio.get_event_loop()
                result = await asyncio.wait_for(
                    loop.run_in_executor(None, task.func, *task.args, **task.kwargs),
                    timeout=task.timeout
                )
            
            # 任务成功
            task.status = TaskStatus.COMPLETED
            task.completed_at = time.time()
            task.result = result
            
            execution_time = task.completed_at - task.started_at
            
            # 创建结果
            task_result = TaskResult(
                task_id=task.id,
                status=TaskStatus.COMPLETED,
                result=result,
                execution_time=execution_time,
                worker_id=worker_id
            )
            
            await self.result_queue.put(task_result)
            
            # 更新统计
            self.stats["completed_tasks"] += 1
            self.stats["total_execution_time"] += execution_time
            
            print(f"      ✅ {worker_id} 完成任务: {task.id} ({execution_time:.3f}s)")
            
        except asyncio.TimeoutError:
            # 任务超时
            await self._handle_task_failure(task, "任务执行超时", worker_id)
            
        except Exception as e:
            # 任务失败
            await self._handle_task_failure(task, str(e), worker_id)
    
    async def _handle_task_failure(self, task: AsyncTask, error: str, worker_id: str):
        """处理任务失败"""
        task.error = error
        
        # 检查是否需要重试
        if task.retry_count < task.max_retries:
            task.retry_count += 1
            task.status = TaskStatus.RETRYING
            task.scheduled_at = time.time() + (2 ** task.retry_count)  # 指数退避
            
            # 重新放入队列
            await self.task_queue.put((task.priority.value, task.id))
            
            self.stats["retry_count"] += 1
            print(f"      🔄 {worker_id} 任务重试: {task.id} (第{task.retry_count}次)")
            
        else:
            # 任务最终失败
            task.status = TaskStatus.FAILED
            task.completed_at = time.time()
            
            execution_time = task.completed_at - task.started_at if task.started_at else 0
            
            task_result = TaskResult(
                task_id=task.id,
                status=TaskStatus.FAILED,
                error=error,
                execution_time=execution_time,
                worker_id=worker_id
            )
            
            await self.result_queue.put(task_result)
            
            self.stats["failed_tasks"] += 1
            print(f"      ❌ {worker_id} 任务失败: {task.id} - {error}")
    
    def get_queue_stats(self) -> Dict[str, Any]:
        """获取队列统计信息"""
        stats = self.stats.copy()
        
        # 计算平均执行时间
        if stats["completed_tasks"] > 0:
            stats["average_execution_time"] = (
                stats["total_execution_time"] / stats["completed_tasks"]
            )
        else:
            stats["average_execution_time"] = 0
        
        # 计算成功率
        total_processed = stats["completed_tasks"] + stats["failed_tasks"]
        if total_processed > 0:
            stats["success_rate"] = f"{stats['completed_tasks']/total_processed*100:.1f}%"
        else:
            stats["success_rate"] = "0%"
        
        stats["queue_size"] = self.task_queue.qsize()
        stats["result_queue_size"] = self.result_queue.qsize()
        stats["active_workers"] = len(self.workers)
        
        return stats
    
    def get_task_status(self, task_id: str) -> Optional[TaskStatus]:
        """获取任务状态"""
        if task_id in self.tasks:
            return self.tasks[task_id].status
        return None

print(f"   ✅ 基础异步任务队列完成")
print(f"      - AsyncTaskQueue: 核心任务队列实现")
print(f"      - 支持优先级、重试、超时机制")
print(f"      - 完整的任务生命周期管理")

print(f"\n✅ 异步任务队列基础完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步任务队列的核心概念和实现")
print(f"   ✓ 学会任务优先级管理和调度策略")
print(f"   ✓ 理解任务重试机制和错误处理")
print(f"   ✓ 能够构建分布式任务队列系统")

### 高级任务队列特性 [⭐⭐进阶]
**知识点说明**：高级任务队列特性是构建生产级异步系统的关键，对于LangChain的企业级部署和高可靠性要求具有重要意义。

**学习要求**：
- 掌握任务持久化和恢复机制
- 学会分布式任务队列设计
- 理解任务监控和调试技术
- 能够实现高可用任务队列系统

**案例要求**：
- 实现完整的分布式任务队列
- 进行任务队列高可用测试
- 应用高级特性解决实际问题
- 验证点：能独立设计企业级任务队列解决方案

In [ ]:
print("\n🚀 高级任务队列特性:")
print("=" * 50)

# 2. 分布式任务队列
print(f"📝 2. 分布式任务队列:")

# 2.1 任务持久化
print(f"\n   🔍 2.1 任务持久化:")

class TaskPersistence:
    """任务持久化管理器"""
    
    def __init__(self, storage_path: str = "./task_storage"):
        self.storage_path = storage_path
        self.task_files = {}
        self.result_files = {}
    
    async def save_task(self, task: AsyncTask) -> bool:
        """保存任务到文件"""
        try:
            filename = f"{self.storage_path}/task_{task.id}.json"
            
            # 序列化任务数据
            task_data = task.to_dict()
            
            # 写入文件（模拟持久化）
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(task_data, f, ensure_ascii=False, indent=2)
            
            self.task_files[task.id] = filename
            return True
            
        except Exception as e:
            print(f"      ❌ 保存任务失败: {e}")
            return False
    
    async def load_task(self, task_id: str, func_registry: Dict[str, Callable]) -> Optional[AsyncTask]:
        """从文件加载任务"""
        try:
            filename = f"{self.storage_path}/task_{task_id}.json"
            
            if not os.path.exists(filename):
                return None
            
            # 读取文件
            with open(filename, 'r', encoding='utf-8') as f:
                task_data = json.load(f)
            
            # 获取函数
            func_name = task_data.get('func_name')
            if func_name not in func_registry:
                print(f"      ⚠️ 函数 {func_name} 未注册")
                return None
            
            # 重建任务
            task = AsyncTask.from_dict(task_data, func_registry[func_name])
            
            self.task_files[task_id] = filename
            return task
            
        except Exception as e:
            print(f"      ❌ 加载任务失败: {e}")
            return None
    
    async def save_result(self, result: TaskResult) -> bool:
        """保存结果到文件"""
        try:
            filename = f"{self.storage_path}/result_{result.task_id}.json"
            
            result_data = asdict(result)
            result_data['status'] = result.status.value
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(result_data, f, ensure_ascii=False, indent=2)
            
            self.result_files[result.task_id] = filename
            return True
            
        except Exception as e:
            print(f"      ❌ 保存结果失败: {e}")
            return False
    
    async def load_pending_tasks(self, func_registry: Dict[str, Callable]) -> List[AsyncTask]:
        """加载所有待处理任务"""
        pending_tasks = []
        
        try:
            if not os.path.exists(self.storage_path):
                os.makedirs(self.storage_path)
                return pending_tasks
            
            # 扫描任务文件
            for filename in os.listdir(self.storage_path):
                if filename.startswith("task_") and filename.endswith(".json"):
                    task_id = filename[5:-5]  # 提取任务ID
                    
                    task = await self.load_task(task_id, func_registry)
                    if task and task.status in [TaskStatus.PENDING, TaskStatus.RETRYING]:
                        pending_tasks.append(task)
            
            print(f"      📂 加载了 {len(pending_tasks)} 个待处理任务")
            return pending_tasks
            
        except Exception as e:
            print(f"      ❌ 加载待处理任务失败: {e}")
            return pending_tasks
    
    async def delete_task(self, task_id: str) -> bool:
        """删除任务文件"""
        try:
            if task_id in self.task_files:
                os.remove(self.task_files[task_id])
                del self.task_files[task_id]
            
            if task_id in self.result_files:
                os.remove(self.result_files[task_id])
                del self.result_files[task_id]
            
            return True
            
        except Exception as e:
            print(f"      ❌ 删除任务文件失败: {e}")
            return False

# 2.2 分布式任务队列
print(f"\n   🌐 2.2 分布式任务队列:")

class DistributedTaskQueue:
    """分布式任务队列"""
    
    def __init__(self, node_id: str, max_workers: int = 5, 
                 storage_path: str = "./distributed_tasks"):
        self.node_id = node_id
        self.max_workers = max_workers
        
        # 基础队列
        self.task_queue = AsyncTaskQueue(
            max_queue_size=1000, 
            max_workers=max_workers
        )
        
        # 持久化
        self.persistence = TaskPersistence(storage_path)
        
        # 函数注册表
        self.func_registry: Dict[str, Callable] = {}
        
        # 节点管理
        self.active_nodes: Dict[str, Dict[str, Any]] = {}
        self.node_heartbeat_interval = 30.0
        
        # 任务分发
        self.task_distribution = {}
        self.load_balancer = LoadBalancer()
        
        # 监控
        self.monitor = TaskMonitor()
    
    def register_function(self, name: str, func: Callable):
        """注册可执行函数"""
        self.func_registry[name] = func
        print(f"      📝 注册函数: {name}")
    
    async def start(self):
        """启动分布式任务队列"""
        print(f"      🚀 启动分布式任务队列节点: {self.node_id}")
        
        # 启动基础队列
        await self.task_queue.start()
        
        # 加载持久化任务
        pending_tasks = await self.persistence.load_pending_tasks(self.func_registry)
        
        # 重新提交待处理任务
        for task in pending_tasks:
            await self.task_queue.task_queue.put((task.priority.value, task.id))
            self.task_queue.tasks[task.id] = task
        
        # 启动心跳
        asyncio.create_task(self._heartbeat_loop())
        
        # 启动监控
        asyncio.create_task(self._monitor_loop())
        
        print(f"      ✅ 分布式任务队列启动完成")
    
    async def stop(self):
        """停止分布式任务队列"""
        print(f"      🛑 停止分布式任务队列节点: {self.node_id}")
        
        await self.task_queue.stop()
        
        print(f"      ✅ 分布式任务队列停止完成")
    
    async def submit_task(self, func_name: str, *args, 
                          priority: TaskPriority = TaskPriority.NORMAL,
                          **kwargs) -> str:
        """提交分布式任务"""
        if func_name not in self.func_registry:
            raise ValueError(f"函数 {func_name} 未注册")
        
        # 创建任务
        task_id = await self.task_queue.submit(
            self.func_registry[func_name],
            *args,
            priority=priority,
            **kwargs
        )
        
        # 持久化任务
        task = self.task_queue.tasks[task_id]
        await self.persistence.save_task(task)
        
        # 记录分发信息
        self.task_distribution[task_id] = {
            "submitted_by": self.node_id,
            "submitted_at": time.time(),
            "func_name": func_name
        }
        
        return task_id
    
    async def get_task_result(self, task_id: str, timeout: float = None) -> Optional[TaskResult]:
        """获取任务结果"""
        # 首先尝试从内存获取
        result = await self.task_queue.get_result(task_id, timeout)
        if result:
            # 持久化结果
            await self.persistence.save_result(result)
            return result
        
        # 尝试从持久化存储获取
        # 这里简化实现，实际应该从文件加载
        return None
    
    async def add_node(self, node_id: str, node_info: Dict[str, Any]):
        """添加节点"""
        self.active_nodes[node_id] = {
            **node_info,
            "last_heartbeat": time.time(),
            "status": "active"
        }
        print(f"      🔗 添加节点: {node_id}")
    
    async def remove_node(self, node_id: str):
        """移除节点"""
        if node_id in self.active_nodes:
            self.active_nodes[node_id]["status"] = "inactive"
            print(f"      ❌ 移除节点: {node_id}")
    
    async def _heartbeat_loop(self):
        """心跳循环"""
        while self.task_queue.running:
            try:
                # 更新自己的心跳
                self.active_nodes[self.node_id] = {
                    "last_heartbeat": time.time(),
                    "status": "active",
                    "queue_size": self.task_queue.task_queue.qsize(),
                    "workers": self.task_queue.max_workers
                }
                
                # 检查其他节点心跳
                current_time = time.time()
                inactive_nodes = []
                
                for node_id, node_info in self.active_nodes.items():
                    if node_id != self.node_id:
                        if current_time - node_info["last_heartbeat"] > self.node_heartbeat_interval * 2:
                            inactive_nodes.append(node_id)
                
                # 移除不活跃节点
                for node_id in inactive_nodes:
                    await self.remove_node(node_id)
                
                await asyncio.sleep(self.node_heartbeat_interval)
                
            except Exception as e:
                print(f"      ❌ 心跳循环错误: {e}")
                await asyncio.sleep(5)
    
    async def _monitor_loop(self):
        """监控循环"""
        while self.task_queue.running:
            try:
                # 收集监控数据
                stats = self.task_queue.get_queue_stats()
                
                # 更新监控器
                self.monitor.update_stats(self.node_id, stats)
                
                # 检查告警条件
                await self._check_alerts(stats)
                
                await asyncio.sleep(10)  # 每10秒监控一次
                
            except Exception as e:
                print(f"      ❌ 监控循环错误: {e}")
                await asyncio.sleep(5)
    
    async def _check_alerts(self, stats: Dict[str, Any]):
        """检查告警条件"""
        # 队列积压告警
        if stats["queue_size"] > 100:
            print(f"      🚨 告警: 任务队列积压严重 ({stats['queue_size']} 个任务)")
        
        # 失败率告警
        total_processed = stats["completed_tasks"] + stats["failed_tasks"]
        if total_processed > 10:
            failure_rate = stats["failed_tasks"] / total_processed
            if failure_rate > 0.1:  # 失败率超过10%
                print(f"      🚨 告警: 任务失败率过高 ({failure_rate*100:.1f}%)")
        
        # 执行时间告警
        if stats["average_execution_time"] > 30.0:
            print(f"      🚨 告警: 平均执行时间过长 ({stats['average_execution_time']:.1f}s)")
    
    def get_cluster_stats(self) -> Dict[str, Any]:
        """获取集群统计信息"""
        return {
            "node_id": self.node_id,
            "active_nodes": len(self.active_nodes),
            "node_list": list(self.active_nodes.keys()),
            "queue_stats": self.task_queue.get_queue_stats(),
            "registered_functions": list(self.func_registry.keys()),
            "monitor_data": self.monitor.get_summary()
        }

# 2.3 负载均衡器
print(f"\n   ⚖️ 2.3 负载均衡器:")

class LoadBalancer:
    """任务负载均衡器"""
    
    def __init__(self):
        self.node_weights: Dict[str, float] = {}
        self.task_assignments: Dict[str, str] = {}  # task_id -> node_id
    
    def select_node(self, available_nodes: List[str], 
                    task_priority: TaskPriority = TaskPriority.NORMAL) -> Optional[str]:
        """选择执行节点"""
        if not available_nodes:
            return None
        
        # 简单的轮询策略
        # 实际应该根据节点负载、任务类型等选择
        
        # 计算权重
        weights = []
        for node_id in available_nodes:
            weight = self.node_weights.get(node_id, 1.0)
            weights.append(weight)
        
        # 选择权重最高的节点
        max_weight_index = weights.index(max(weights))
        selected_node = available_nodes[max_weight_index]
        
        # 更新权重（降低选中节点的权重）
        self.node_weights[selected_node] = max(0.1, weight - 0.1)
        
        # 恢复其他节点权重
        for node_id in available_nodes:
            if node_id != selected_node:
                self.node_weights[node_id] = min(2.0, self.node_weights.get(node_id, 1.0) + 0.05)
        
        return selected_node
    
    def assign_task(self, task_id: str, node_id: str):
        """分配任务到节点"""
        self.task_assignments[task_id] = node_id
    
    def get_node_load(self, node_id: str) -> int:
        """获取节点负载"""
        return sum(1 for assigned_node in self.task_assignments.values() 
                  if assigned_node == node_id)

# 2.4 任务监控器
print(f"\n   📊 2.4 任务监控器:")

class TaskMonitor:
    """任务监控器"""
    
    def __init__(self):
        self.metrics_history: Dict[str, List[Dict[str, Any]]] = {}
        self.alerts: List[Dict[str, Any]] = []
        self.max_history = 1000
    
    def update_stats(self, node_id: str, stats: Dict[str, Any]):
        """更新统计信息"""
        timestamp = time.time()
        
        if node_id not in self.metrics_history:
            self.metrics_history[node_id] = []
        
        metrics = {
            "timestamp": timestamp,
            **stats
        }
        
        self.metrics_history[node_id].append(metrics)
        
        # 限制历史记录长度
        if len(self.metrics_history[node_id]) > self.max_history:
            self.metrics_history[node_id] = self.metrics_history[node_id][-self.max_history:]
    
    def add_alert(self, alert_type: str, message: str, severity: str = "warning"):
        """添加告警"""
        alert = {
            "type": alert_type,
            "message": message,
            "severity": severity,
            "timestamp": time.time()
        }
        
        self.alerts.append(alert)
        
        # 限制告警历史
        if len(self.alerts) > 100:
            self.alerts = self.alerts[-100:]
    
    def get_summary(self) -> Dict[str, Any]:
        """获取监控摘要"""
        return {
            "monitored_nodes": len(self.metrics_history),
            "total_alerts": len(self.alerts),
            "recent_alerts": self.alerts[-10:],
            "node_metrics": {
                node_id: len(history) 
                for node_id, history in self.metrics_history.items()
            }
        }

# 创建存储目录
import os
if not os.path.exists("./distributed_tasks"):
    os.makedirs("./distributed_tasks")

print(f"   ✅ 分布式任务队列完成")
print(f"      - TaskPersistence: 任务持久化管理")
print(f"      - DistributedTaskQueue: 分布式任务队列")
print(f"      - LoadBalancer: 负载均衡器")
print(f"      - TaskMonitor: 任务监控器")

print(f"\n✅ 高级任务队列特性完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握任务持久化和恢复机制")
print(f"   ✓ 学会分布式任务队列设计")
print(f"   ✓ 理解任务监控和调试技术")
print(f"   ✓ 能够实现高可用任务队列系统")

### LangChain任务队列应用 [⭐⭐进阶]
**知识点说明**：LangChain任务队列应用是掌握AI系统异步处理的关键，对于LangChain的大规模模型推理和数据处理具有重要意义。

**学习要求**：
- 掌握LangChain任务的异步处理模式
- 学会AI模型推理的队列管理
- 理解LangChain工作流的异步编排
- 能够构建高性能LangChain处理系统

**案例要求**：
- 实现完整的LangChain任务队列系统
- 进行AI工作流的异步编排
- 应用任务队列解决LangChain实际问题
- 验证点：能独立设计高性能AI异步处理系统

In [ ]:
print("\n🤖 LangChain任务队列应用:")
print("=" * 50)

# 3. LangChain异步任务处理
print(f"📝 3. LangChain异步任务处理:")

# 3.1 AI模型推理任务
print(f"\n   🔍 3.1 AI模型推理任务:")

@dataclass
class LLMInferenceTask:
    """LLM推理任务"""
    task_id: str
    prompt: str
    model_name: str
    max_tokens: int = 1000
    temperature: float = 0.7
    context: Dict[str, Any] = field(default_factory=dict)
    priority: TaskPriority = TaskPriority.NORMAL
    
@dataclass
class LLMInferenceResult:
    """LLM推理结果"""
    task_id: str
    response: str
    model_name: str
    tokens_used: int
    processing_time: float
    confidence: float
    worker_id: str

class LangChainTaskQueue:
    """LangChain专用任务队列"""
    
    def __init__(self, max_workers: int = 3):
        self.base_queue = AsyncTaskQueue(max_workers=max_workers)
        self.model_pools: Dict[str, List[str]] = {}
        self.request_limits: Dict[str, Dict[str, float]] = {}
        self.inference_stats = {
            "total_requests": 0,
            "successful_requests": 0,
            "failed_requests": 0,
            "total_tokens": 0,
            "total_response_time": 0.0
        }
    
    async def start(self):
        """启动LangChain任务队列"""
        await self.base_queue.start()
        
        # 初始化模型池
        self.model_pools = {
            "gpt-4": [f"gpt-4-worker-{i}" for i in range(2)],
            "gpt-3.5-turbo": [f"gpt-3.5-worker-{i}" for i in range(3)],
            "claude-3": [f"claude-3-worker-{i}" for i in range(2)]
        }
        
        # 初始化请求限制
        self.request_limits = {
            "gpt-4": {"rate_limit": 10.0, "last_request": 0},
            "gpt-3.5-turbo": {"rate_limit": 20.0, "last_request": 0},
            "claude-3": {"rate_limit": 15.0, "last_request": 0}
        }
        
        print(f"      🚀 LangChain任务队列启动")
    
    async def stop(self):
        """停止LangChain任务队列"""
        await self.base_queue.stop()
        print(f"      🛑 LangChain任务队列停止")
    
    async def submit_inference(self, llm_task: LLMInferenceTask) -> str:
        """提交LLM推理任务"""
        # 检查模型可用性
        if llm_task.model_name not in self.model_pools:
            raise ValueError(f"不支持的模型: {llm_task.model_name}")
        
        # 检查请求限制
        await self._check_rate_limit(llm_task.model_name)
        
        # 创建推理函数
        inference_func = self._create_inference_function(llm_task)
        
        # 提交任务
        task_id = await self.base_queue.submit(
            inference_func,
            priority=llm_task.priority,
            timeout=60.0  # LLM推理超时时间
        )
        
        # 存储任务映射
        self.base_queue.tasks[task_id].context = {
            "llm_task": llm_task,
            "original_task_id": llm_task.task_id
        }
        
        self.inference_stats["total_requests"] += 1
        
        print(f"      📤 提交LLM推理任务: {llm_task.task_id} (模型: {llm_task.model_name})")
        return task_id
    
    async def get_inference_result(self, task_id: str, timeout: float = None) -> Optional[LLMInferenceResult]:
        """获取推理结果"""
        result = await self.base_queue.get_result(task_id, timeout)
        
        if result and result.status == TaskStatus.COMPLETED:
            return result.result
        elif result and result.status == TaskStatus.FAILED:
            self.inference_stats["failed_requests"] += 1
            return None
        
        return None
    
    def _create_inference_function(self, llm_task: LLMInferenceTask) -> Callable:
        """创建推理函数"""
        async def inference_function():
            return await self._perform_llm_inference(llm_task)
        
        return inference_function
    
    async def _perform_llm_inference(self, llm_task: LLMInferenceTask) -> LLMInferenceResult:
        """执行LLM推理"""
        start_time = time.time()
        worker_id = f"llm-worker-{random.randint(1, 100)}"
        
        # 模拟不同模型的推理时间
        if llm_task.model_name == "gpt-4":
            processing_time = random.uniform(2.0, 5.0)
        elif llm_task.model_name == "gpt-3.5-turbo":
            processing_time = random.uniform(1.0, 3.0)
        elif llm_task.model_name == "claude-3":
            processing_time = random.uniform(1.5, 4.0)
        else:
            processing_time = random.uniform(1.0, 2.0)
        
        await asyncio.sleep(processing_time)
        
        # 模拟推理结果
        response_length = min(llm_task.max_tokens, random.randint(100, 800))
        response = f"Model {llm_task.model_name} response to '{llm_task.prompt[:50]}...' "
        response += f"with {response_length} tokens of comprehensive analysis. "
        response += f"Temperature {llm_task.temperature} applied for creativity."
        
        tokens_used = response_length + len(llm_task.prompt.split())
        confidence = random.uniform(0.8, 0.95)
        
        actual_time = time.time() - start_time
        
        # 更新统计
        self.inference_stats["successful_requests"] += 1
        self.inference_stats["total_tokens"] += tokens_used
        self.inference_stats["total_response_time"] += actual_time
        
        return LLMInferenceResult(
            task_id=llm_task.task_id,
            response=response,
            model_name=llm_task.model_name,
            tokens_used=tokens_used,
            processing_time=actual_time,
            confidence=confidence,
            worker_id=worker_id
        )
    
    async def _check_rate_limit(self, model_name: str):
        """检查请求限制"""
        if model_name not in self.request_limits:
            return
        
        limit_info = self.request_limits[model_name]
        current_time = time.time()
        
        # 检查是否需要等待
        time_since_last = current_time - limit_info["last_request"]
        min_interval = 60.0 / limit_info["rate_limit"]  # 每秒请求数转换为间隔
        
        if time_since_last < min_interval:
            wait_time = min_interval - time_since_last
            await asyncio.sleep(wait_time)
        
        limit_info["last_request"] = time.time()
    
    def get_inference_stats(self) -> Dict[str, Any]:
        """获取推理统计"""
        stats = self.inference_stats.copy()
        
        # 计算平均值
        if stats["successful_requests"] > 0:
            stats["average_tokens"] = stats["total_tokens"] / stats["successful_requests"]
            stats["average_response_time"] = stats["total_response_time"] / stats["successful_requests"]
        else:
            stats["average_tokens"] = 0
            stats["average_response_time"] = 0
        
        # 计算成功率
        total = stats["total_requests"]
        if total > 0:
            stats["success_rate"] = f"{stats['successful_requests']/total*100:.1f}%"
        else:
            stats["success_rate"] = "0%"
        
        stats["queue_stats"] = self.base_queue.get_queue_stats()
        
        return stats

# 3.2 LangChain工作流编排
print(f"\n   🔄 3.2 LangChain工作流编排:")

@dataclass
class WorkflowStep:
    """工作流步骤"""
    step_id: str
    name: str
    func: Callable
    depends_on: List[str] = field(default_factory=list)
    retry_count: int = 3
    timeout: float = 30.0

@dataclass
class Workflow:
    """工作流定义"""
    workflow_id: str
    name: str
    steps: List[WorkflowStep]
    created_at: float = field(default_factory=time.time)

@dataclass
class WorkflowExecution:
    """工作流执行"""
    execution_id: str
    workflow_id: str
    step_results: Dict[str, Any] = field(default_factory=dict)
    step_status: Dict[str, TaskStatus] = field(default_factory=dict)
    started_at: float = field(default_factory=time.time)
    completed_at: Optional[float] = None
    status: TaskStatus = TaskStatus.PENDING

class LangChainWorkflowEngine:
    """LangChain工作流引擎"""
    
    def __init__(self, task_queue: LangChainTaskQueue):
        self.task_queue = task_queue
        self.workflows: Dict[str, Workflow] = {}
        self.executions: Dict[str, WorkflowExecution] = {}
        self.workflow_stats = {
            "total_workflows": 0,
            "completed_workflows": 0,
            "failed_workflows": 0,
            "total_steps": 0,
            "completed_steps": 0
        }
    
    def register_workflow(self, workflow: Workflow):
        """注册工作流"""
        self.workflows[workflow.workflow_id] = workflow
        print(f"      📝 注册工作流: {workflow.name} ({len(workflow.steps)} 个步骤)")
    
    async def execute_workflow(self, workflow_id: str, 
                               context: Dict[str, Any] = None) -> str:
        """执行工作流"""
        if workflow_id not in self.workflows:
            raise ValueError(f"工作流 {workflow_id} 未注册")
        
        workflow = self.workflows[workflow_id]
        execution_id = str(uuid.uuid4())
        
        # 创建执行实例
        execution = WorkflowExecution(
            execution_id=execution_id,
            workflow_id=workflow_id
        )
        
        self.executions[execution_id] = execution
        self.workflow_stats["total_workflows"] += 1
        self.workflow_stats["total_steps"] += len(workflow.steps)
        
        print(f"      🚀 开始执行工作流: {workflow.name}")
        
        # 执行工作流
        try:
            await self._execute_workflow_steps(workflow, execution, context or {})
            
            execution.status = TaskStatus.COMPLETED
            execution.completed_at = time.time()
            
            self.workflow_stats["completed_workflows"] += 1
            print(f"      ✅ 工作流执行完成: {workflow.name} ({execution.completed_at - execution.started_at:.3f}s)")
            
        except Exception as e:
            execution.status = TaskStatus.FAILED
            execution.completed_at = time.time()
            
            self.workflow_stats["failed_workflows"] += 1
            print(f"      ❌ 工作流执行失败: {workflow.name} - {e}")
        
        return execution_id
    
    async def _execute_workflow_steps(self, workflow: Workflow, 
                                      execution: WorkflowExecution, 
                                      context: Dict[str, Any]):
        """执行工作流步骤"""
        # 构建依赖图
        dependency_graph = self._build_dependency_graph(workflow)
        
        # 按依赖顺序执行步骤
        executed_steps = set()
        
        while len(executed_steps) < len(workflow.steps):
            # 找到可以执行的步骤
            ready_steps = []
            
            for step in workflow.steps:
                if step.step_id not in executed_steps:
                    # 检查依赖是否完成
                    dependencies_met = all(
                        dep in executed_steps 
                        for dep in step.depends_on
                    )
                    
                    if dependencies_met:
                        ready_steps.append(step)
            
            if not ready_steps:
                raise Exception("工作流依赖循环或未满足的依赖")
            
            # 并发执行就绪的步骤
            step_tasks = []
            
            for step in ready_steps:
                execution.step_status[step.step_id] = TaskStatus.RUNNING
                
                step_task = asyncio.create_task(
                    self._execute_workflow_step(step, execution, context)
                )
                step_tasks.append(step_task)
            
            # 等待所有步骤完成
            step_results = await asyncio.gather(*step_tasks, return_exceptions=True)
            
            # 处理结果
            for i, result in enumerate(step_results):
                step = ready_steps[i]
                
                if isinstance(result, Exception):
                    execution.step_status[step.step_id] = TaskStatus.FAILED
                    raise result
                else:
                    execution.step_results[step.step_id] = result
                    execution.step_status[step.step_id] = TaskStatus.COMPLETED
                    executed_steps.add(step.step_id)
                    self.workflow_stats["completed_steps"] += 1
                    
                    print(f"      ✅ 步骤完成: {step.name}")
    
    async def _execute_workflow_step(self, step: WorkflowStep, 
                                     execution: WorkflowExecution, 
                                     context: Dict[str, Any]) -> Any:
        """执行单个工作流步骤"""
        try:
            # 准备步骤参数
            step_context = {
                **context,
                "step_results": execution.step_results,
                "workflow_execution": execution
            }
            
            # 执行步骤
            if asyncio.iscoroutinefunction(step.func):
                result = await asyncio.wait_for(
                    step.func(step_context),
                    timeout=step.timeout
                )
            else:
                loop = asyncio.get_event_loop()
                result = await asyncio.wait_for(
                    loop.run_in_executor(None, step.func, step_context),
                    timeout=step.timeout
                )
            
            return result
            
        except asyncio.TimeoutError:
            raise Exception(f"步骤 {step.name} 执行超时")
        except Exception as e:
            raise Exception(f"步骤 {step.name} 执行失败: {e}")
    
    def _build_dependency_graph(self, workflow: Workflow) -> Dict[str, List[str]]:
        """构建依赖图"""
        graph = {}
        
        for step in workflow.steps:
            graph[step.step_id] = step.depends_on.copy()
        
        return graph
    
    def get_workflow_stats(self) -> Dict[str, Any]:
        """获取工作流统计"""
        stats = self.workflow_stats.copy()
        
        # 计算成功率
        total = stats["total_workflows"]
        if total > 0:
            stats["workflow_success_rate"] = f"{stats['completed_workflows']/total*100:.1f}%"
        else:
            stats["workflow_success_rate"] = "0%"
        
        # 计算步骤成功率
        total_steps = stats["total_steps"]
        if total_steps > 0:
            stats["step_success_rate"] = f"{stats['completed_steps']/total_steps*100:.1f}%"
        else:
            stats["step_success_rate"] = "0%"
        
        stats["registered_workflows"] = len(self.workflows)
        stats["active_executions"] = len([
            e for e in self.executions.values() 
            if e.status == TaskStatus.RUNNING
        ])
        
        return stats

print(f"   ✅ LangChain任务队列应用完成")
print(f"      - LangChainTaskQueue: LangChain专用任务队列")
print(f"      - LangChainWorkflowEngine: 工作流编排引擎")
print(f"      - 支持LLM推理任务和复杂工作流")

print(f"\n✅ LangChain任务队列应用完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握LangChain任务的异步处理模式")
print(f"   ✓ 学会AI模型推理的队列管理")
print(f"   ✓ 理解LangChain工作流的异步编排")
print(f"   ✓ 能够构建高性能LangChain处理系统")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.7 异步任务队列 [⭐⭐进阶]**
- ✅ 掌握异步任务队列的核心概念和实现
- ✅ 学会任务优先级管理和调度策略
- ✅ 理解任务重试机制和错误处理
- ✅ 能够构建分布式任务队列系统
- ✅ 掌握任务持久化和恢复机制
- ✅ 学会分布式任务队列设计
- ✅ 理解任务监控和调试技术
- ✅ 能够实现高可用任务队列系统
- ✅ 掌握LangChain任务的异步处理模式
- ✅ 学会AI模型推理的队列管理
- ✅ 理解LangChain工作流的异步编排
- ✅ 能够构建高性能LangChain处理系统
- ✅ 能独立设计和实现任务队列解决方案
- ✅ 能独立设计企业级任务队列解决方案
- ✅ 能独立设计高性能AI异步处理系统

### 🎯 与LangChain学习的关联

**异步任务队列重要性**：
- 任务队列优化LangChain的批量处理能力
- 优先级管理提升LangChain的任务调度效率
- 重试机制确保LangChain的可靠性
- 分布式队列支持LangChain的水平扩展
- 工作流编排增强LangChain的复杂任务处理

**实际应用场景**：
- LangChain的大规模模型推理队列
- LangChain的文档处理和分析流水线
- LangChain的多步骤AI工作流自动化
- LangChain的实时对话和响应系统
- LangChain的分布式AI服务部署

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习复杂任务队列的设计和实现
   - 掌握分布式系统的协调和通信机制
   - 学习大规模任务调度和负载均衡算法

2. **扩展学习**：
   - 了解消息队列系统（RabbitMQ、Kafka）
   - 学习容器化和微服务架构
   - 探索云原生任务调度平台

3. **实际应用**：
   - 构建生产级LangChain任务处理系统
   - 开发分布式AI模型推理平台
   - 实现企业级工作流自动化系统

### 🔧 常见错误与注意事项

1. **任务队列阻塞**：
   ```python
   # 错误：同步任务阻塞异步队列
   async def bad_task_processor():
       while True:
           task = await queue.get()
           result = sync_heavy_computation(task.data)  # 阻塞事件循环
           await process_result(result)
   
   # 正确：在线程池中执行同步任务
   async def good_task_processor():
       loop = asyncio.get_event_loop()
       while True:
           task = await queue.get()
           result = await loop.run_in_executor(
               None, sync_heavy_computation, task.data
           )
           await process_result(result)
   ```

2. **任务丢失风险**：
   ```python
   # 错误：没有持久化机制
   class BadTaskQueue:
       def __init__(self):
           self.tasks = []  # 内存存储，重启丢失
       
       async def add_task(self, task):
           self.tasks.append(task)  # 系统崩溃时丢失
   
   # 正确：实现持久化机制
   class GoodTaskQueue:
       def __init__(self, storage_backend):
           self.memory_tasks = []
           self.storage = storage_backend
       
       async def add_task(self, task):
           # 先持久化，再添加到内存
           await self.storage.save_task(task)
           self.memory_tasks.append(task)
   ```

3. **无限重试循环**：
   ```python
   # 错误：没有重试限制或退避策略
   async def bad_retry_handler(task):
       while True:
           try:
               await execute_task(task)
               break
           except Exception:
               await asyncio.sleep(0.1)  # 立即重试，可能导致风暴
   
   # 正确：实现指数退避和重试限制
   async def good_retry_handler(task, max_retries=3):
       for attempt in range(max_retries):
           try:
               await execute_task(task)
               return
           except Exception as e:
               if attempt == max_retries - 1:
                   raise e
               
               # 指数退避
               wait_time = (2 ** attempt) + random.uniform(0, 1)
               await asyncio.sleep(wait_time)
   ```

4. **资源耗尽问题**：
   ```python
   # 错误：无限制任务提交
   async def bad_task_submitter():
       while True:
           task = create_task()
           await queue.submit(task)  # 可能导致内存溢出
   
   # 正确：实现背压机制
   async def good_task_submitter():
       while True:
           # 检查队列大小
           if queue.size() < MAX_QUEUE_SIZE:
               task = create_task()
               await queue.submit(task)
           else:
               # 队列满时等待或丢弃任务
               await asyncio.sleep(1)
   ```

5. **任务依赖管理混乱**：
   ```python
   # 错误：没有依赖检查
   async def bad_workflow_executor():
       # 并发执行所有步骤，忽略依赖关系
       await asyncio.gather(*[execute_step(step) for step in steps])
   
   # 正确：构建依赖图并按序执行
   async def good_workflow_executor():
       dependency_graph = build_dependency_graph(steps)
       execution_order = topological_sort(dependency_graph)
       
       for step_group in execution_order:
           # 并发执行无依赖的步骤组
           await asyncio.gather(*[execute_step(step) for step in step_group])
   ```

6. **监控和可观测性不足**：
   ```python
   # 错误：没有监控机制
   class BadTaskQueue:
       async def process_tasks(self):
           while True:
               task = await self.get_task()
               await self.execute_task(task)  # 没有日志和监控
   
   # 正确：实现全面的监控
   class GoodTaskQueue:
       def __init__(self):
           self.metrics = TaskMetrics()
           self.logger = setup_logger()
       
       async def process_tasks(self):
           while True:
               task = await self.get_task()
               
               start_time = time.time()
               try:
                   result = await self.execute_task(task)
                   self.metrics.record_success(task, time.time() - start_time)
                   self.logger.info(f"Task {task.id} completed successfully")
               except Exception as e:
                   self.metrics.record_failure(task, e)
                   self.logger.error(f"Task {task.id} failed: {e}")
   ```

### 🌐 性能优化建议

**任务队列优化**：
- 根据任务特性选择合适的队列类型
- 实现智能任务调度和负载均衡
- 使用批处理减少系统开销
- 实现动态工作进程调整

**分布式优化**：
- 优化网络通信和数据序列化
- 实现本地性调度减少网络延迟
- 使用一致性哈希分配任务
- 实现故障转移和自动恢复

**LangChain优化**：
- 实现模型特定的优化策略
- 使用缓存减少重复推理
- 实现请求合并和批处理
- 优化工作流依赖和并行度

---

**🎉 恭喜完成异步任务队列学习！**

你已经全面掌握了异步任务队列的核心技术，能够系统性地进行任务队列设计、分布式部署、工作流编排等，为LangChain智能应用提供了强大的任务处理基础。

## 🚀 下一步学习预告

**第六节：异步编程 - 进行中** 🔄
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ✅
- 6.6 并发编程模式 ✅
- 6.7 异步任务队列 ✅
- 6.8 异步性能优化 ⏳
- 6.9 异步最佳实践 ⏳

**继续第六节：异步编程**
- 下一个：6.8 异步性能优化
- 深入学习异步性能分析和优化技术
- 掌握异步系统的性能调优方法

**后续章节预告**：
- Web开发技术
- 项目工程实践

继续加油，异步任务队列已经掌握！准备深入学习异步性能优化！